In [13]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn.model_selection as sk
from sklearn.preprocessing import StandardScaler

In [14]:
#load data
df=pd.read_csv("myeod.csv")
df.replace([np.inf], np.nan)
df.dropna(axis=0, how='all')
df.head()

,Ticker,Date,AdjOpen,AdjHigh,AdjLow,AdjClose,AdjVol,returns_adj_close,upmove,downmove,...,ma.40,ma.80,macd.10v5,macd.20v10,macd.40v20,macd.80v40,percent.K.5,percent.K.14,percent.K.30,percent.K.60
0,INDB,19960216,0.916667,0.950000,0.916667,0.916667,0.065677,0.964912,0.0,0.0,...,0.935041,0.936923,0.019301,0.251480,0.409883,0.174791,0.000000,0.000000,0.000000,0.215847
1,INDB,19960220,0.950000,0.950000,0.916667,0.950000,0.090868,1.036364,0.0,0.0,...,0.935771,0.937246,-0.347682,0.089301,0.321271,0.136999,1.000000,1.000000,0.495049,0.442718
2,INDB,19960606,0.953846,0.969231,0.953846,0.961539,0.090676,0.992063,0.0,0.0,...,0.954149,0.933659,-0.096926,-0.101673,-0.271853,-0.730347,0.480769,0.480769,0.506579,0.734992
3,BANF,19930422,0.875588,0.901725,0.875588,0.901725,0.076594,1.029851,0.0,0.0,...,0.914304,0.894553,0.078036,0.495841,0.280168,-0.375729,1.000000,1.000000,0.500000,0.500000
4,INDB,19960607,0.969231,0.969231,0.953846,0.953846,0.091701,0.992000,0.0,0.0,...,0.954134,0.934157,0.041618,-0.051579,-0.245460,-0.712054,0.000000,0.000000,0.342105,0.681991


In [15]:
df.shape

(692842, 37)

In [51]:
X_train, X_cv_test, Y_train, Y_cv_test = sk.train_test_split(
    df.drop(["Ticker","Date","returns_adj_close"], axis=1).astype(np.float32),
    pd.DataFrame(df["returns_adj_close"].astype(np.float32)),
    test_size=0.40,
    train_size =0.60,
    random_state = 42)

In [52]:
X_cv, X_test, Y_cv, Y_test = sk.train_test_split(
    X_cv_test,
    Y_cv_test,
    test_size=0.50,
    random_state = 42)

In [53]:
X_train.iloc[135989 ,11]

4.3388128

In [108]:
learning_rate = 0.00005
training_epochs = 150

In [210]:
number_of_inputs = 34
number_of_output = 1

In [211]:
tf.reset_default_graph()

In [212]:
layer_1_nodes = 300
layer_2_nodes = 200
layer_3_nodes = 10

In [213]:
# Input Layer
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape=(None, number_of_inputs))

In [214]:
with tf.variable_scope('layer_1'):
    weights = tf.get_variable(name = "weights1", shape = [number_of_inputs, layer_1_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name = "biases1", shape = [layer_1_nodes], initializer = tf.zeros_initializer())
    layer_1_output = tf.nn.sigmoid( tf.matmul(X, weights) + biases)

In [215]:
with tf.variable_scope('layer_2'):
    weights = tf.get_variable(name = "weights2", shape = [layer_1_nodes, layer_2_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name = "biases2", shape = [layer_2_nodes], initializer = tf.zeros_initializer())
    layer_2_output = tf.nn.sigmoid( tf.matmul(layer_1_output, weights) + biases)

In [216]:
with tf.variable_scope('layer_3'):
    weights = tf.get_variable(name = "weights3", shape = [layer_2_nodes, layer_3_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name = "biases3", shape = [layer_3_nodes], initializer = tf.zeros_initializer())
    layer_3_output = tf.nn.sigmoid( tf.matmul(layer_2_output, weights) + biases)

In [217]:
with tf.variable_scope('output'):
    weights = tf.get_variable(name = "weights4", shape = [layer_3_nodes, number_of_output], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name = "biases4", shape = [number_of_output], initializer = tf.zeros_initializer())
    output_layer = tf.matmul(layer_3_output, weights) + biases

In [218]:
with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, 1))
    cost = tf.reduce_mean(tf.squared_difference(output_layer,Y))

In [219]:
with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [220]:
with tf.variable_scope("logging"):
    tf.summary.scalar("current_cost",cost)
    summary=tf.summary.merge_all()

In [221]:
print ("Layer 1 nodes: ", layer_1_nodes)
print ("Layer 2 nodes: ", layer_2_nodes)
print ("Layer 3 nodes: ", layer_3_nodes)

print("\n============================================================================\n============================================================================\n")


with tf.Session() as session:

    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.global_variables_initializer())

    # Create log file writers to record training progress.
    # We'll store training and testing log data separately.
    training_writer = tf.summary.FileWriter('./logs/200/training-'+str(layer_1_nodes)+'-'+str(layer_2_nodes)+'-'+str(layer_3_nodes), session.graph)
    cv_writer = tf.summary.FileWriter('./logs/200/cv-'+str(layer_1_nodes)+'-'+str(layer_2_nodes)+'-'+str(layer_3_nodes), session.graph)

    # Run the optimizer over and over to train the network.
    # One epoch is one full run through the training data set.
    for epoch in range(training_epochs):

        # Feed in the training data and do one step of neural network training
        session.run(optimizer, feed_dict={X: X_train, Y: Y_train})

        # Every 5 training steps, log our progress
        if epoch % 5 == 0:
            # Get the current accuracy scores by running the "cost" operation on the training and test data sets
            training_cost, training_summary = session.run([cost, summary], feed_dict={X: X_train, Y:Y_train})
            cv_cost, cv_summary = session.run([cost, summary], feed_dict={X: X_cv, Y: Y_cv})

            # Write the current training status to the log files (Which we can view with TensorBoard)
            training_writer.add_summary(training_summary, epoch)
            cv_writer.add_summary(cv_summary, epoch)

            # Print the current training status to the screen
            print("Epoch {}: \t Training Cost: {}  \t CV Cost: {}".format(epoch,training_cost,cv_cost))

    # Training is now complete!

    # Get the final accuracy scores by running the "cost" operation on the training and test data sets
    final_training_cost = session.run(cost, feed_dict={X: X_train, Y: Y_train})
    final_cv_cost = session.run(cost, feed_dict={X: X_cv, Y: Y_cv})
    final_testing_cost=session.run(cost, feed_dict={X: X_test, Y: Y_test})
    print("\n============================================================================\n============================================================================\n")
    print("Final Training cost:\t {}".format(final_training_cost))
    print("Final CV cost      :\t {}".format(final_cv_cost))
    print("Final Testing cost :\t {}".format(final_testing_cost))
    

Layer 1 nodes:  300
Layer 2 nodes:  200
Layer 3 nodes:  10


Epoch 0: 	 Training Cost: 0.5105771422386169  	 CV Cost: 0.5105352997779846
Epoch 5: 	 Training Cost: 0.4539581537246704  	 CV Cost: 0.45391884446144104
Epoch 10: 	 Training Cost: 0.40128934383392334  	 CV Cost: 0.4012509882450104
Epoch 15: 	 Training Cost: 0.3529181480407715  	 CV Cost: 0.35288000106811523
Epoch 20: 	 Training Cost: 0.30904144048690796  	 CV Cost: 0.3090059459209442
Epoch 25: 	 Training Cost: 0.26970869302749634  	 CV Cost: 0.2696716785430908
Epoch 30: 	 Training Cost: 0.23481318354606628  	 CV Cost: 0.234778493642807
Epoch 35: 	 Training Cost: 0.20413875579833984  	 CV Cost: 0.20410452783107758
Epoch 40: 	 Training Cost: 0.17737381160259247  	 CV Cost: 0.17734089493751526
Epoch 45: 	 Training Cost: 0.154154971241951  	 CV Cost: 0.15412163734436035
Epoch 50: 	 Training Cost: 0.13409043848514557  	 CV Cost: 0.13405777513980865
Epoch 55: 	 Training Cost: 0.11679684370756149  	 CV Cost: 0.1167646050453186
Epoch